# H3 indicator calculation sql

H3 indicator calculation sqls using the LG methodology v0.1: https://docs.google.com/document/d/1s1rINj-YVDQ36Vu-Q3T1sc55hcVLROJclaaELRF9tck/edit#

List of indicators:

- Farm level indicators:
    - Water use
    - Unustainable water use
    - Land use
    - Greenhouse gas emission (in progress)

- Landscape level indicators
    - Deforestation risk 
    - Climate risk from land use
    - Biodiversity risk from land use change (in progress)


## Farm level:

### Water use

Impact (c,g) = volume (c,g) * blwft (c,g)

### Unsustainable water use

Same as above but where BWS >= 0.4

Impact (c,g) = volume (c,g) * blwft (c,g)* %water stress in location


### Land use

Impact (c,g) = Volume(c,g) * SUM(HA(c,pixel)) / SUM(Prod (c,pixel))

## Landscape level

### Deforestation risk

impact(c,g) = sum(defkernel def(pixel) * prod (pixel)) / sum(prod(pixel)) * land use impact / ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))

### Climate risk from land use

Impact (c,g) = (sum(carbon*def kernel * prod (pixel)) / sum(prod)) * land use impact / ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))

### Update 10/22: Include satelligence indicators:

#### 1. Satelligence deforestation:

#### 2. Satelligence deforestation risk:

In [1]:
# import libraries
from psycopg2.pool import ThreadedConnectionPool

import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import json
import numpy as np

In [2]:
#set env
## env file for gcs upload
env_path = ".env"
with open(env_path) as f:
    env = {}
    for line in f:
        env_key, _val = line.split("=", 1)
        env_value = _val.split("\n")[0]
        env[env_key] = env_value
        
#list(env.keys())

# set conexion to local ddbb
postgres_thread_pool = ThreadedConnectionPool(
    1, 
    50,
    host=env['API_POSTGRES_HOST'],
    port=env['API_POSTGRES_PORT'],
    user=env['API_POSTGRES_USERNAME'],
    password=env['API_POSTGRES_PASSWORD']
)

#get list of sourcing records to iterate:
conn = postgres_thread_pool.getconn()
cursor = conn.cursor()

In [3]:
def psql(query):
    try:
        cursor.execute(query)
        return cursor.fetchall()
    except Exception as e:
        conn.rollback()
        print(e)

In [5]:
SQL_GET_H3_UNCOMPACT_GEO_REGION = """
CREATE OR REPLACE FUNCTION get_h3_uncompact_geo_region(geo_region_id uuid, h3_resolution int)
RETURNS TABLE (h3index h3index) AS 
$$
    SELECT h3_uncompact(geo_region."h3Compact"::h3index[], h3_resolution) h3index
    FROM geo_region WHERE geo_region.id = geo_region_id
$$ 
LANGUAGE SQL;
"""

SQL_SUM_H3_GRID_OVER_GEO_REGION = SQL_GET_H3_UNCOMPACT_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_h3_grid_over_georegion(
    geo_region_id uuid, 
    h3_resolution int,
    h3_table_name varchar, 
    h3_column_name varchar
)
RETURNS float AS
$$
    DECLARE
        sum float;
    BEGIN
        EXECUTE format(
            'SELECT sum(h3grid.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid ON h3grid.h3index = geo_region.h3index;
            ', h3_column_name, h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL = """
CREATE OR REPLACE FUNCTION get_h3_table_column_for_material(material_id uuid, h3_data_type material_to_h3_type_enum)
RETURNS TABLE (h3_table_name varchar, h3_column_name varchar, h3_resolution int) AS
$$
    SELECT h3_data."h3tableName", h3_data."h3columnName", h3_data."h3resolution"
    FROM h3_data
        INNER JOIN material_to_h3 ON material_to_h3."h3DataId" = h3_data.id
    WHERE material_to_h3."materialId" = material_id 
        AND material_to_h3.type = h3_data_type
    LIMIT 1;
$$
LANGUAGE SQL;
"""

SQL_SUM_MATERIAL_OVER_GEO_REGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+SQL_GET_H3_TABLE_COLUMN_FOR_MATERIAL+"""
CREATE OR REPLACE FUNCTION sum_material_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        h3_table_name varchar;
        h3_column_name varchar;
        h3_resolution integer;
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO h3_table_name, h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        SELECT sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""



# Weighted mean total cropland area per pixel ((sum(all_harvest area(pixel) * crop_prod (pixel)) / sum(crop_prod))
SQL_SUM_WEIGHTED_CROPLAND_AREA_ALL = """CREATE OR REPLACE FUNCTION sum_h3_weighted_cropland_area_all(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        agri_ha_h3_table_name varchar := 'h3_grid_spam2010v2r0_global_ha';
        agri_ha_h3_column_name varchar := 'spam2010V2R0GlobalHAcofA'; --update with new column name;
        chickens_ha_table_name varchar := 'h3_grid_chickens_glo_ha';
        chickens_ha_column_name varchar := 'glw32010ChickensHa';
        goats_ha_table_name varchar := 'h3_grid_goats_glo_lsu';
        goats_ha_column_name varchar := 'glw32010GoatsLsu';
        horses_ha_table_name varchar := 'h3_grid_horses_glo_ha';
        horses_ha_column_name varchar := 'glw32010HorsesHa';
        pigs_ha_table_name varchar := 'h3_grid_pigs_glo_ha';
        pigs_ha_column_name varchar := 'glw32010PigsHa';
        sheep_ha_table_name varchar := 'h3_grid_sheep_glo_ha';
        sheep_ha_column_name varchar := 'glw32010SheepHa';
        sum float;
    BEGIN
        --Get h3data table name and column for a given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        
        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * 
            (h3grid_all_ha.%I +  h3grid_chickens_ha.%I + h3grid_goats_ha.%I + h3grid_horses_ha.%I + h3grid_pigs_ha.%I + h3grid_sheep_ha.%I))
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_all_ha ON h3grid_all_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_chickens_ha ON h3grid_chickens_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_goats_ha ON h3grid_goats_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_horses_ha ON h3grid_horses_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_pigs_ha ON h3grid_pigs_ha.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_sheep_ha ON h3grid_sheep_ha.h3index = geo_region.h3index;
                ', material_h3_column_name, agri_ha_h3_column_name, chickens_ha_column_name, 
                goats_ha_column_name, horses_ha_column_name, pigs_ha_column_name, sheep_ha_column_name, 
                material_h3_table_name, agri_ha_h3_table_name, chickens_ha_table_name, goats_ha_table_name,
                horses_ha_table_name, pigs_ha_table_name, sheep_ha_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;     
$$
LANGUAGE plpgsql;
"""
SQL_SUM_WEIGHTED_CROPLAND_AREA = """CREATE OR REPLACE FUNCTION sum_h3_weighted_cropland_area(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        agri_ha_h3_table_name varchar := 'h3_grid_spam2010v2r0_global_ha';
        agri_ha_h3_column_name varchar := 'spam2010V2R0GlobalHAcofA'; --update with new column name;
        sum float;
    BEGIN
        --Get h3data table name and column for a given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        
        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * h3grid_all_ha.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_all_ha ON h3grid_all_ha.h3index = geo_region.h3index
                ', material_h3_column_name, agri_ha_h3_column_name,material_h3_table_name, agri_ha_h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;     
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_deforestation_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        deforestation_h3_table_name varchar := 'h3_grid_deforestation_global'; --update with new deforestation table
        deforestation_h3_column_name varchar := 'hansenLoss2019';
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum table column over region
        EXECUTE format(
            'SELECT sum(h3grid_mat.%I * h3grid_def.%I)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region
                    INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                    INNER JOIN %I h3grid_def ON h3grid_def.h3index = geo_region.h3index;
            ', material_h3_column_name, deforestation_h3_column_name, material_h3_table_name, deforestation_h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""

SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION = """
CREATE OR REPLACE FUNCTION sum_weighted_carbon_over_georegion(
    geo_region_id uuid, 
    material_id uuid,
    h3_data_type material_to_h3_type_enum
)
RETURNS float AS
$$
    DECLARE
        material_h3_table_name varchar;
        material_h3_column_name varchar;
        h3_resolution integer;
        carbon_h3_table_name varchar := 'h3_grid_carbon_global'; -- update this table with new one
        carbon_h3_column_name varchar := 'earthstat2000GlobalHectareEmissions';--update this table with new one
        sum float;

    BEGIN
        -- Get h3data table name and column name for given material --
        SELECT * INTO material_h3_table_name, material_h3_column_name, h3_resolution
        FROM get_h3_table_column_for_material(material_id, h3_data_type);

        -- Sum deforestation times carbon where material is produced --
        EXECUTE format(
                'SELECT sum(h3grid_mat.%I * h3grid_carbon.%I) --carbon has been already multiplied by the deforestation
                    FROM get_h3_uncompact_geo_region($1, $2) geo_region
                        INNER JOIN %I h3grid_mat ON h3grid_mat.h3index = geo_region.h3index
                        INNER JOIN %I h3grid_carbon ON h3grid_carbon.h3index = geo_region.h3index;', 
                material_h3_column_name, 
                carbon_h3_column_name,
                material_h3_table_name,
                carbon_h3_table_name
            )
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;
"""
SQL_GET_PERCENTAGE_WATER_STRESS_AREA = """
CREATE OR REPLACE FUNCTION get_percentage_water_stress_area(
geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE
        aqueduct_h3_table_name varchar := 'h3_grid_aqueduct_global';
        aqueduct_h3_column_name varchar := 'bwsCat';
        h3_resolution integer := 6;
        percentage float;

    BEGIN
        EXECUTE format(
            'SELECT reduced.ws_area/ reduced.g_area as percentage
            FROM
                (SELECT 
                    sum(case when aqueduct.%I > 3 then 1 else 0 end) ws_area, count(aqueduct.%I) g_area
                FROM get_h3_uncompact_geo_region($1, $2) geo_region
                INNER JOIN %I aqueduct ON aqueduct.h3index = geo_region.h3index) reduced
            WHERE reduced.g_area > 0;',
            aqueduct_h3_column_name,
            aqueduct_h3_column_name,
            aqueduct_h3_table_name
                )
            USING geo_region_id, h3_resolution
            INTO percentage;
        RETURN percentage;
    END;
$$
LANGUAGE plpgsql;
"""


SQL_GET_BLWF_IMPACT = """
CREATE OR REPLACE FUNCTION get_blwf_impact(
    adminRegionId uuid,
    material_id uuid)
RETURNS float AS
$$
    DECLARE
        indicator_coeficient_table varchar := 'indicator_coefficient';
        indicator_coeficient_value varchar := 'value';
        value float;
    BEGIN
       
        EXECUTE format(
            'SELECT reduced.%I
                FROM (
                    SELECT ind_coef.%I
                    FROM %I ind_coef
                    WHERE (ind_coef."adminRegionId" = $1 AND ind_coef."materialId" = $2)
                    OR (ind_coef."materialId" = $2 AND ind_coef."adminRegionId" IS NULL )  
                    )reduced
                WHERE reduced.%I is not null;',
                indicator_coeficient_value,
                indicator_coeficient_value,
                indicator_coeficient_table,
                indicator_coeficient_value
                )
            USING adminRegionId, material_id
            INTO value;
        RETURN value;
    END;
$$
LANGUAGE plpgsql;
"""

In [5]:
## FUTURE IMPROVEMENTS - REMOVE HARDCODED TABLE NAMES
## NEW QUERIES FOR SATELLIGENCE DATA
#SQL_GET_H3_TABLE_COLUMN_FOR_INDICATOR = """
#CREATE OR REPLACE FUNCTION get_h3_table_column_for_indicator(nameCode varchar)
#RETURNS TABLE (h3_table_name varchar, h3_column_name varchar, h3_resolution int, h3_year int) AS
#$$
#    SELECT h3_data."h3tableName", h3_data."h3columnName", h3_data."h3resolution", h3_data."year"
#    FROM h3_data
#        INNER JOIN  "indicator" indicator ON  indicator.id = h3_data."indicatorId" 
#    WHERE indicator."nameCode" = nameCode
#    LIMIT 1;
#$$
#LANGUAGE SQL;
#"""
#
#SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+SQL_GET_H3_TABLE_COLUMN_FOR_INDICATOR+"""
#CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_over_georegion(
#    geo_region_id uuid,
#    nameCode varchar,
#    year int
#)
#RETURNS float AS
#$$
#    DECLARE 
#        h3_table_name varchar;
#        h3_column_name varchar;
#        h3_resolution integer;
#        h3_year integer;
#        sum float;
#    BEGIN
#        -- Get h3data table name and column name for a given indicator
#        SELECT * INTO h3_table_name, h3_column_name, h3_resolution, h3_year
#        FROM get_h3_table_column_for_indicator(nameCode);
#        
#        -- Sum table column over region
#        SELECT case when h3_year = year 
#        then sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
#        else 0 end
#        INTO sum;
#        RETURN sum;
#    END;      
#$$
#LANGUAGE plpgsql;
#"""
#


## Same h3 resolution:

In [6]:
SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_over_georegion(
    geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE 
        h3_table_name varchar := 'h3_grid_carbon_global';
        h3_column_name varchar := 'earthstat2000GlobalHectareEmissions' ;
        h3_resolution integer := 6;
        sum float;
    BEGIN      
        -- Sum table column over region
        SELECT sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;      
$$
LANGUAGE plpgsql;
"""

SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION = SQL_SUM_H3_GRID_OVER_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_risk_over_georegion(
    geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE 
        h3_table_name varchar := 'h3_grid_carbon_global';
        h3_column_name varchar := 'earthstat2000GlobalHectareEmissions' ;
        h3_resolution integer := 6;
        sum float;
    BEGIN      
        -- Sum table column over region
        SELECT sum_h3_grid_over_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;      
$$
LANGUAGE plpgsql;
"""

In [7]:
#%%time
sourcing_locations = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    +SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    +SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    + """
    SELECT
        id,
        sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
        sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
        --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
        sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
        sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon,
        --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
        --get_blwf_impact("adminRegionId", "materialId") as raw_water
        sum_satelligence_deforestation_over_georegion("geoRegionId") as sat_deforestation,
        sum_satelligence_deforestation_risk_over_georegion("geoRegionId") as sat_deforestation_risk  
    FROM
        sourcing_location
    """, conn)

sourcing_locations

,id,production,harvested_area,raw_deforestation,raw_carbon,sat_deforestation,sat_deforestation_risk
0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,134.073011,134.073011
1,ea1556f9-27a4-4f0f-a308-bf2fc6a0ea3e,1.928499e+06,2.834290e+06,1.848665e+06,1.468524e+07,64926.983735,64926.983735
2,5e394935-c9e1-4957-8183-6377a229ae8a,7.144756e+01,4.013908e+01,4.183139e+01,2.770092e+01,329967.843794,329967.843794
3,6896d22f-5be4-4e32-900d-799a1aeba022,NaN,NaN,NaN,NaN,48232.469682,48232.469682
4,b476ee6c-e43d-4707-bfe3-f624e174d002,1.305452e+06,8.676576e+05,1.043980e+06,1.559114e+06,124271.253037,124271.253037
...,...,...,...,...,...,...,...
6041,579a669e-32d1-42b2-b380-51e31584a791,7.643713e+06,1.917111e+06,1.839570e+04,9.106267e+06,21081.305574,21081.305574
6042,ac075d1c-9c12-4c62-83bf-29b04cf0acfc,1.059660e+06,1.905122e+05,9.060586e+05,3.394628e+06,19995.649010,19995.649010
6043,82b6930f-6b10-46e1-96b2-af538e97de07,7.546656e+05,1.803208e+05,2.654272e+05,2.707200e+06,978.216466,978.216466
6044,41e43c4d-244b-4016-962f-7711ef086c22,4.002255e+05,1.288886e+05,7.803750e+04,5.633177e+05,383.645780,383.645780


In [8]:
sourcing_records = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CROPLAND_AREA \
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION \
    + SQL_GET_PERCENTAGE_WATER_STRESS_AREA \
    + SQL_GET_BLWF_IMPACT \
    + SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    + """
    SELECT
        sr.id,
        sr.tonnage,
        sr.year,
        sl.id,
        sl.production,
        sl.harvested_area,
        --sl.weighted_all_ha,
        sl.raw_deforestation,
        sl.raw_carbon,
        --sl.water_stress_perct
        --sl.raw_water  
        case when sr.year=2021 then sl.sat_deforestation else 0 end sat_deforestation_2021,
        case when sr.year=2021 then sl.sat_deforestation_risk else 0 end sat_deforestation_risk
    FROM
        sourcing_records sr
        INNER JOIN
            (
                SELECT
                    id,
                    "geoRegionId",
                    sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
                    sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
                    --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
                    sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
                    sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon,
                    --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
                    --get_blwf_impact("adminRegionId", "materialId") as raw_water
                    sum_satelligence_deforestation_over_georegion("geoRegionId") as sat_deforestation,
                    sum_satelligence_deforestation_risk_over_georegion("geoRegionId") as sat_deforestation_risk  
                FROM
                    sourcing_location
            ) as sl
            on sr."sourcingLocationId" = sl.id
""", conn)

sourcing_records['land_per_ton'] = sourcing_records['harvested_area'] / sourcing_records['production']
##Weighted mean total cropland area
#sourcing_records['weighted_total_cropland_area'] = sourcing_records['weighted_all_ha'] / sourcing_records['production']
#sourcing_records['deforestation_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_deforestation'] / sourcing_records['weighted_total_cropland_area'], 0)
#sourcing_records['carbon_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_carbon'] / sourcing_records['weighted_total_cropland_area'], 0)
#
#
#sourcing_records['land_use'] = sourcing_records['land_per_ton'] * sourcing_records['tonnage']
#sourcing_records['deforestation'] = sourcing_records['deforestation_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['carbon_loss'] = sourcing_records['carbon_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['water_use'] = sourcing_records['raw_water'] * sourcing_records['tonnage']
#sourcing_records['unsustainable_water_use'] = sourcing_records['water_use'] * sourcing_records['water_stress_perct']
#sourcing_records.to_csv('test_impact_calc.csv')


sourcing_records

,id,tonnage,year,id,production,harvested_area,raw_deforestation,raw_carbon,sat_deforestation_2021,sat_deforestation_risk,land_per_ton
0,75d25991-7d15-4f36-9f9a-95dd127371e0,2400.0,2010,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
1,e445629b-3e38-4516-aa9e-6023b2303255,2424.0,2011,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
2,1a3d56e4-821e-448b-91b1-d22b35977927,2448.0,2012,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
3,032eb005-fd3e-4fc5-9dee-cf922e7ab611,2472.0,2013,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
4,7ca403f1-d5d5-444f-baed-a7ba2eea3148,2497.0,2014,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
...,...,...,...,...,...,...,...,...,...,...,...
18511,4373d929-d4f3-4322-8483-683f94fdaa55,2264.5,2020,579a669e-32d1-42b2-b380-51e31584a791,7.643713e+06,1.917111e+06,1.839570e+04,9.106267e+06,0.0,0.0,2.508089e-01
18512,94363d79-533f-48cd-8bb7-37f4ce903f9c,2154.0,2020,ac075d1c-9c12-4c62-83bf-29b04cf0acfc,1.059660e+06,1.905122e+05,9.060586e+05,3.394628e+06,0.0,0.0,1.797861e-01
18513,c8d32111-5974-499b-9e82-dfbdcbee24df,1712.5,2020,82b6930f-6b10-46e1-96b2-af538e97de07,7.546656e+05,1.803208e+05,2.654272e+05,2.707200e+06,0.0,0.0,2.389413e-01
18514,bed644fe-54a0-4b75-bdea-27be7f2419a6,1491.5,2020,41e43c4d-244b-4016-962f-7711ef086c22,4.002255e+05,1.288886e+05,7.803750e+04,5.633177e+05,0.0,0.0,3.220399e-01


## Different h3 resolution

In [9]:
SQL_SUM_H3_GRID_PARENT_OVER_GEO_REGION = SQL_GET_H3_UNCOMPACT_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_h3_grid_over_parent_georegion(
    geo_region_id uuid, 
    h3_resolution int,
    h3_table_name varchar, 
    h3_column_name varchar
)
RETURNS float AS
$$
    DECLARE
        sum float;
    BEGIN
        EXECUTE format(
            'SELECT sum(parent_ind.v)
                FROM
                    get_h3_uncompact_geo_region($1, $2) geo_region --res6
                INNER JOIN 
                    (SELECT h3_to_parent(indTable.h3index,6) h3index, sum(indTable.%I) v 
                    FROM %I as indTable
                    GROUP BY h3index) parent_ind                    
                ON parent_ind.h3index = geo_region.h3index; --res11
            ', h3_column_name, h3_table_name)
            USING geo_region_id, h3_resolution
            INTO sum;
        RETURN sum;
    END;
$$
LANGUAGE plpgsql;

"""

In [10]:
SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION_PARENT = SQL_SUM_H3_GRID_PARENT_OVER_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_over_georegion_parent(
    geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE 
        h3_table_name varchar := 'h3_grid_carbon_global';
        h3_column_name varchar := 'earthstat2000GlobalHectareEmissions' ;
        h3_resolution integer := 6;
        sum float;
    BEGIN      
        -- Sum table column over region
        SELECT sum_h3_grid_over_parent_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;      
$$
LANGUAGE plpgsql;
"""

SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION_PARENT = SQL_SUM_H3_GRID_PARENT_OVER_GEO_REGION+"""
CREATE OR REPLACE FUNCTION sum_satelligence_deforestation_risk_over_georegion_parent(
    geo_region_id uuid
)
RETURNS float AS
$$
    DECLARE 
        h3_table_name varchar := 'h3_grid_carbon_global';
        h3_column_name varchar := 'earthstat2000GlobalHectareEmissions' ;
        h3_resolution integer := 6;
        sum float;
    BEGIN      
        -- Sum table column over region
        SELECT sum_h3_grid_over_parent_georegion(geo_region_id, h3_resolution, h3_table_name, h3_column_name) 
        INTO sum;
        RETURN sum;
    END;      
$$
LANGUAGE plpgsql;
"""


In [ ]:
%%time
sourcing_locations = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    +SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    +SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION_PARENT\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION_PARENT\
    + """
    SELECT
        id,
        sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
        sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
        --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
        sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
        sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon,
        --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
        --get_blwf_impact("adminRegionId", "materialId") as raw_water
        sum_satelligence_deforestation_over_georegion_parent("geoRegionId") as sat_deforestation,
        sum_satelligence_deforestation_risk_over_georegion_parent("geoRegionId") as sat_deforestation_risk  
    FROM
        sourcing_location
    """, conn)

sourcing_locations

In [6]:
#%%time
sourcing_locations = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION
    +SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    +SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    + """
    SELECT
        id,
        sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
        sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
        --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
        sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
        sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon,
        --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
        --get_blwf_impact("adminRegionId", "materialId") as raw_water
        sum_satelligence_deforestation_over_georegion("geoRegionId") as sat_deforestation,
        sum_satelligence_deforestation_risk_over_georegion("geoRegionId") as sat_deforestation_risk  
    FROM
        sourcing_location
    """, conn)

sourcing_locations

,id,production,harvested_area,raw_deforestation,raw_carbon,sat_deforestation,sat_deforestation_risk
0,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,134.073011,134.073011
1,ea1556f9-27a4-4f0f-a308-bf2fc6a0ea3e,1.928499e+06,2.834290e+06,1.848665e+06,1.468524e+07,64926.983735,64926.983735
2,5e394935-c9e1-4957-8183-6377a229ae8a,7.144756e+01,4.013908e+01,4.183139e+01,2.770092e+01,329967.843794,329967.843794
3,6896d22f-5be4-4e32-900d-799a1aeba022,NaN,NaN,NaN,NaN,48232.469682,48232.469682
4,b476ee6c-e43d-4707-bfe3-f624e174d002,1.305452e+06,8.676576e+05,1.043980e+06,1.559114e+06,124271.253037,124271.253037
...,...,...,...,...,...,...,...
6041,579a669e-32d1-42b2-b380-51e31584a791,7.643713e+06,1.917111e+06,1.839570e+04,9.106267e+06,21081.305574,21081.305574
6042,ac075d1c-9c12-4c62-83bf-29b04cf0acfc,1.059660e+06,1.905122e+05,9.060586e+05,3.394628e+06,19995.649010,19995.649010
6043,82b6930f-6b10-46e1-96b2-af538e97de07,7.546656e+05,1.803208e+05,2.654272e+05,2.707200e+06,978.216466,978.216466
6044,41e43c4d-244b-4016-962f-7711ef086c22,4.002255e+05,1.288886e+05,7.803750e+04,5.633177e+05,383.645780,383.645780


In [9]:
sourcing_records = pd.read_sql_query(
    SQL_SUM_MATERIAL_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CROPLAND_AREA \
    + SQL_SUM_WEIGHTED_DEFORESTATION_OVER_GEO_REGION \
    + SQL_SUM_WEIGHTED_CARBON_OVER_GEO_REGION \
    + SQL_GET_PERCENTAGE_WATER_STRESS_AREA \
    + SQL_GET_BLWF_IMPACT \
    + SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_RISK_OVER_GEOREGION\
    +SQL_SUM_SATELLIGENCE_DEFORESTATION_OVER_GEOREGION\
    + """
    SELECT
        sr.id,
        sr.tonnage,
        sr.year,
        sl.id,
        sl.production,
        sl.harvested_area,
        --sl.weighted_all_ha,
        sl.raw_deforestation,
        sl.raw_carbon,
        --sl.water_stress_perct
        --sl.raw_water  
        case when sr.year=2021 then sl.sat_deforestation else 0 end sat_deforestation_2021,
        case when sr.year=2021 then sl.sat_deforestation_risk else 0 end sat_deforestation_risk
    FROM
        sourcing_records sr
        INNER JOIN
            (
                SELECT
                    id,
                    "geoRegionId",
                    sum_material_over_georegion("geoRegionId", "materialId", 'producer') as production,
                    sum_material_over_georegion("geoRegionId", "materialId", 'harvest') as harvested_area,
                    --sum_h3_weighted_cropland_area("geoRegionId", "materialId", 'producer') as weighted_all_ha,
                    sum_weighted_deforestation_over_georegion("geoRegionId", "materialId", 'producer') as raw_deforestation,
                    sum_weighted_carbon_over_georegion("geoRegionId", "materialId", 'producer') as raw_carbon,
                    --get_percentage_water_stress_area("geoRegionId") as water_stress_perct
                    --get_blwf_impact("adminRegionId", "materialId") as raw_water
                    sum_satelligence_deforestation_over_georegion("geoRegionId") as sat_deforestation,
                    sum_satelligence_deforestation_risk_over_georegion("geoRegionId") as sat_deforestation_risk  
                FROM
                    sourcing_location
            ) as sl
            on sr."sourcingLocationId" = sl.id
""", conn)

sourcing_records['land_per_ton'] = sourcing_records['harvested_area'] / sourcing_records['production']
##Weighted mean total cropland area
#sourcing_records['weighted_total_cropland_area'] = sourcing_records['weighted_all_ha'] / sourcing_records['production']
#sourcing_records['deforestation_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_deforestation'] / sourcing_records['weighted_total_cropland_area'], 0)
#sourcing_records['carbon_per_ha_landuse'] = np.where(sourcing_records['weighted_total_cropland_area'] >0, sourcing_records['raw_carbon'] / sourcing_records['weighted_total_cropland_area'], 0)
#
#
#sourcing_records['land_use'] = sourcing_records['land_per_ton'] * sourcing_records['tonnage']
#sourcing_records['deforestation'] = sourcing_records['deforestation_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['carbon_loss'] = sourcing_records['carbon_per_ha_landuse'] * sourcing_records['land_use']
#sourcing_records['water_use'] = sourcing_records['raw_water'] * sourcing_records['tonnage']
#sourcing_records['unsustainable_water_use'] = sourcing_records['water_use'] * sourcing_records['water_stress_perct']
#sourcing_records.to_csv('test_impact_calc.csv')


sourcing_records

,id,tonnage,year,id,production,harvested_area,raw_deforestation,raw_carbon,sat_deforestation_2021,sat_deforestation_risk,land_per_ton
0,75d25991-7d15-4f36-9f9a-95dd127371e0,2400.0,2010,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
1,e445629b-3e38-4516-aa9e-6023b2303255,2424.0,2011,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
2,1a3d56e4-821e-448b-91b1-d22b35977927,2448.0,2012,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
3,032eb005-fd3e-4fc5-9dee-cf922e7ab611,2472.0,2013,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
4,7ca403f1-d5d5-444f-baed-a7ba2eea3148,2497.0,2014,49db98c9-61f0-4faa-b979-cae487fe2c95,9.364061e-39,1.745669e-17,3.465075e-39,4.775168e-39,0.0,0.0,1.864222e+21
...,...,...,...,...,...,...,...,...,...,...,...
18511,4373d929-d4f3-4322-8483-683f94fdaa55,2264.5,2020,579a669e-32d1-42b2-b380-51e31584a791,7.643713e+06,1.917111e+06,1.839570e+04,9.106267e+06,0.0,0.0,2.508089e-01
18512,94363d79-533f-48cd-8bb7-37f4ce903f9c,2154.0,2020,ac075d1c-9c12-4c62-83bf-29b04cf0acfc,1.059660e+06,1.905122e+05,9.060586e+05,3.394628e+06,0.0,0.0,1.797861e-01
18513,c8d32111-5974-499b-9e82-dfbdcbee24df,1712.5,2020,82b6930f-6b10-46e1-96b2-af538e97de07,7.546656e+05,1.803208e+05,2.654272e+05,2.707200e+06,0.0,0.0,2.389413e-01
18514,bed644fe-54a0-4b75-bdea-27be7f2419a6,1491.5,2020,41e43c4d-244b-4016-962f-7711ef086c22,4.002255e+05,1.288886e+05,7.803750e+04,5.633177e+05,0.0,0.0,3.220399e-01
